In [1]:
from transformers import BertModel, BertTokenizer
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка предобученной модели и токенизатора BERT
model_name = 'bert-base-uncased'  # можно выбрать другие модели, например, 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [262]:
import pandas as pd

In [296]:
dt_avas = pd.read_excel(r'/content/Регламент по пунктам AVAS_v3 (1).xlsx')
dt_wipe = pd.read_excel(r'/content/WINDSCREEN WIPER AND WASHER.xlsx')

In [309]:
dt = pd.concat([dt_avas,dt_wipe],ignore_index = True)

In [310]:
dt

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,NaN,NaN
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,NaN
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,NaN


In [311]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        # Используем эмбеддинг токена [CLS], который представляет всё предложение
        return outputs.last_hidden_state[:, 0, :].cpu().numpy()


In [312]:
import spacy
from nltk.corpus import stopwords
import nltk
import re
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def tokenizers(x):
  cleaned_text = re.sub(r'[^\w\s]', '', x)
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(cleaned_text)
  tokens = [token.text for token in doc]
  return tokens


def to_chto(text,stop_words):
  cleaned_text = tokenizers(text)
  dano = [word.lower() for word in cleaned_text if word.lower() not in stop_words]
  text = ' '.join(dano)
  return text
def calculate_cosine_similarity(embedding,target_embedding_2d):
    return cosine_similarity(embedding, target_embedding_2d).flatten()[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [313]:
#dt['new'] = dt['под-подпункт'].fillna(dt['подпункт']).fillna(' ')

In [314]:
dt.fillna('',inplace=True)

In [315]:
#dt['preobraz'] = dt['new'].apply(lambda x: to_chto(x,stop_words))
#dt['emb'] = dt['preobraz'].apply(lambda x: get_embedding(x.lower()))

In [316]:
dk = dt.drop(['emb','preobraz','new'],axis = 1)

In [317]:
target = 'Car emit sound out_27, because EV operate quietly, and this sound helps to alert pedestrians of your oncoming vehicle'
target_embedding = get_embedding(to_chto(target,stop_words))

In [319]:
id = dk.index.to_list()

for i in ['Глава','Под глава', 'подпункт', 'под-подпункт']:

    df = dk.loc[id]  # Используйте loc для работы с индексами

    # Применение функций для вычисления эмбеддингов и схожести
    df['emb'] = (df[i]
                  .apply(lambda x: to_chto(x, stop_words))
                  .apply(lambda x: get_embedding(x.lower())))
    display(df)

    df['similarity'] = df['emb'].apply(lambda x: calculate_cosine_similarity(x, target_embedding))

    display(df)

    # Отображение DataFrame
    # Поиск максимального значения схожести
    max_similarity = df['similarity'].max()

    # Обновление списка индексов для следующей итерации
    id = df[df['similarity'] == max_similarity].index.to_list()


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175..."


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb,similarity
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.741677


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-0.21406586, 0.32362592, 0.30832857, -0.1190..."
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266..."


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb,similarity
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-0.21406586, 0.32362592, 0.30832857, -0.1190...",0.760239
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,"[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.758248


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-1.0533974, -0.41769162, -0.3749478, -0.1958..."


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb,similarity
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-1.0533974, -0.41769162, -0.3749478, -0.1958...",0.381193


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-1.0533974, -0.41769162, -0.3749478, -0.1958..."


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,emb,similarity
0,6.1,6,AVAS,1,General specifications\nFor the purpose of thi...,,,"[[-1.0533974, -0.41769162, -0.3749478, -0.1958...",0.381193


In [284]:
#dt['embd_под'] = (dt['Под глава'].apply(lambda x: to_chto(x,stop_words))).apply(lambda x: get_embedding(x.lower()))

In [286]:
#text = ['driving below approximately 32 km/h or while driving in reverse','Car emit sound out_27, because EV operate quietly, and this sound helps to alert pedestrians of your oncoming vehicle','Pedestrians know that car is near']

In [ ]:
#reg_predl = dict()
#for predl in text:
  #emd = to_chto(predl,stop_words)
  #target_embedding = get_embedding(emd)
  #dt['similarity'] = dt['emb'].apply(lambda x: calculate_cosine_similarity(x,target_embedding))
  #top_1 = dt.sort_values(by = 'similarity',ascending = False).reset_index(drop = True)
  #reg_predl[predl] = top_1['new'].to_list()[0]

In [228]:
#for predl in text:
  #emd = to_chto(predl,stop_words)
  #target_embedding = get_embedding(emd)
  #dt['similarity'] = dt['emb'].apply(lambda x: calculate_cosine_similarity(x,target_embedding))
  #top_1 = dt.sort_values(by = 'similarity',ascending = False).reset_index(drop = True)
  #df = top_1[['index','similarity','Под глава','подпункт','под-подпункт']]
  #df['Offer'] = predl
  #df.to_excel(f'{predl[:20]}.xlsx',index=False)
  #reg_predl[predl] = top_1['new'].to_list()[0]

<ipython-input-228-190b80d73dd8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Offer'] = predl
<ipython-input-228-190b80d73dd8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Offer'] = predl
<ipython-input-228-190b80d73dd8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie